In [1]:
import torch
from torch import nn
## torch version
print(torch.__version__)

1.13.1+cu116


In [2]:
import random

# generate samples (a, b)
# 0 <= a <= 1, 0 <= b <= 1
# right ( positive ) --> 15
# left ( negative ) --> 15

torch.manual_seed(42)

def rand_sample_generation(pos_n, neg_n):
    
    # y = 0.7x + b
    # --> 0.7x - y + b = 0
    # --> 0.7x1 - x2 + b = 0
    m = 0.7
    b = 0.3

    pos = []
    neg = []
    while True:
        x_tmp = random.random() * 100 - 50
        y_tmp = random.random() * 100 - 50
        # +-1 for label
        if m * x_tmp - y_tmp + b > 0 and len(pos) < pos_n:
            pos.append([x_tmp, y_tmp, 1])
        elif m * x_tmp - y_tmp + b < 0 and len(neg) < neg_n:
            neg.append([x_tmp, y_tmp, -1])

        if len(pos) == pos_n and len(neg) == neg_n:
            break

    # print("pos :", len(pos), pos)
    # print("neg :", len(neg), neg)

    sample = pos + neg
    # print("sample :", len(sample), sample)

    # shuffle pos neg
    random.shuffle(sample)

    # return sample in tensor
    sample_tensor = torch.tensor(sample, dtype=torch.float32)
    return sample_tensor

In [3]:
def need_update(v1, v2, label):
    res = label * torch.dot(v1, v2)
    if res < 0:
        return True
    else:
        return False

def PLA(data, weight, max_iter=1000, lr=1):
    iter_count = 0
    for i in range(max_iter):
        update_count = 0
        for j in range(len(data)):
            data_vec = torch.cat((
                data[j][:2],
                torch.tensor([1], dtype=torch.float32)
            ))
            label = data[j][2:3]
            if need_update(data_vec, weight, label):
                weight = weight + lr * label * data_vec
                update_count = update_count + 1
        
        # bread when we don't neet to update
        if update_count == 0:
            break
        
        iter_count = iter_count + 1

    # return final weights and total iterations
    return weight, iter_count

In [4]:
# initialize weight
weight = torch.rand(3)
print(weight)

tensor([0.8823, 0.9150, 0.3829])


In [5]:
# generate random sample
data = rand_sample_generation(15, 15)
print(data)

tensor([[-15.3536,   7.4616,  -1.0000],
        [ 18.4288,  31.9370,  -1.0000],
        [-41.9092, -26.4187,  -1.0000],
        [-15.4115, -46.9166,   1.0000],
        [-10.4020,   7.6098,  -1.0000],
        [-43.3205,   1.4967,  -1.0000],
        [-35.1359,  -2.1730,  -1.0000],
        [-20.7053, -31.7365,   1.0000],
        [  1.2750, -28.0854,   1.0000],
        [-39.2225, -32.2348,   1.0000],
        [-16.8428, -26.4752,   1.0000],
        [ 10.0873,  26.9734,  -1.0000],
        [  3.9953, -21.0184,   1.0000],
        [  7.2118, -31.5616,   1.0000],
        [-36.9033, -44.2527,   1.0000],
        [ 12.6171, -48.7071,   1.0000],
        [ 19.9786,  33.7099,  -1.0000],
        [-18.4480,  37.6435,  -1.0000],
        [-43.7406,  36.4861,  -1.0000],
        [ 36.7934,  -7.0908,   1.0000],
        [-14.4023,  -7.6041,  -1.0000],
        [ 41.5265,   8.0071,   1.0000],
        [ 23.1404,  45.1217,  -1.0000],
        [ 40.9922,  33.7690,  -1.0000],
        [ 17.7193,  27.8783,  -1.0000],


In [6]:
fin_weight, iter_count = PLA(data, weight)
print("fin_weight:", fin_weight)
print("iter_count:", iter_count)

fin_weight: tensor([ 49.8511, -64.4137,  -1.6171])
iter_count: 2


In [51]:
# three time to calculate average
iter_avg = 0
for i in range(3):
    data = rand_sample_generation(15, 15)
    fin_weight, iter_count = PLA(data, weight)
    print("i:", i, ", iter_count:", iter_count)
    iter_avg = iter_avg + iter_count
iter_avg = iter_avg / 3
print("iter_avg:", iter_avg)

i: 0 , iter_count: 25
i: 1 , iter_count: 1
i: 2 , iter_count: 6
iter_avg: 10.666666666666666


In [8]:
# generate random sample
data = rand_sample_generation(1000, 1000)
print(data)

tensor([[-48.4270,  27.7707,  -1.0000],
        [-40.5495,  20.1939,  -1.0000],
        [ 47.2934, -40.4861,   1.0000],
        ...,
        [ -8.0677, -10.7784,   1.0000],
        [ -2.7212,  39.2475,  -1.0000],
        [  4.6934,  -9.2533,   1.0000]])


In [9]:
# pocket algorithm
# random weight
def pocket_alg(data, max_iter=1000, lr=1):

    weight = torch.rand(3)

    # declare final weight
    fin_weight = weight
    
    # declare err count min
    err_count_min = len(data) + 1

    for i in range(max_iter):
        err_count = 0
        for j in range(len(data)):
            data_vec = torch.cat((
                data[j][:2],
                torch.tensor([1], dtype=torch.float32)
            ))
            label = data[j][2:3]
            if need_update(data_vec, weight, label):
                weight = weight + lr * label * data_vec
                err_count = err_count + 1
        if err_count_min > err_count:
            err_count_min = err_count
            fin_weight = weight
        # print("fin_weight:", fin_weight, ", err_count_min:", err_count_min)

    err_rate = err_count / len(data)
    
    # return fin_weight, err_rate
    return fin_weight, err_rate

In [10]:
import time

# initialize weight
weight = torch.rand(3)
print(weight)

tensor([0.9593, 0.3904, 0.6009])


In [11]:
# try PLA
tic = time.process_time()
fin_weight, iter_count = PLA(data, weight)
toc = time.process_time()
print ("--> PLA: " + str(1000*(toc - tic)) + "ms")

# try POCKET
tic = time.process_time()
fin_weight, err_rate = pocket_alg(data, max_iter=200)
toc = time.process_time()
print ("--> POCKET: " + str(1000*(toc - tic)) + "ms")
print("pocket accuracy:", 1-err_rate)

--> PLA: 1540.9111080000005ms
--> POCKET: 16692.379519ms
pocket accuracy: 1.0


In [12]:
# mislabeled
def rand_sample_generation_mis50(pos_n, neg_n):
    
    # y = 0.7x + b
    # --> 0.7x - y + b = 0
    # --> 0.7x1 - x2 + b = 0
    m = 0.7
    b = 0.3

    pos = []
    neg = []
    while True:
        x_tmp = random.random() * 100 - 50
        y_tmp = random.random() * 100 - 50
        # +-1 for label
        if m * x_tmp - y_tmp + b > 0 and len(pos) < pos_n - 50:
            pos.append([x_tmp, y_tmp, 1])
        elif m * x_tmp - y_tmp + b < 0 and len(neg) < neg_n - 50:
            neg.append([x_tmp, y_tmp, -1])

        if len(pos) == pos_n - 50 and len(neg) == neg_n - 50:
            break

    # mislabeled data
    while True:
        x_tmp = random.random() * 100 - 50
        y_tmp = random.random() * 100 - 50
        # +-1 for label
        if m * x_tmp - y_tmp + b > 0 and len(pos) < pos_n:
            pos.append([x_tmp, y_tmp, -1])
        elif m * x_tmp - y_tmp + b < 0 and len(neg) < neg_n:
            neg.append([x_tmp, y_tmp, 1])

        if len(pos) == pos_n and len(neg) == neg_n:
            break

    # print("pos :", len(pos))
    # print("neg :", len(neg))

    sample = pos + neg
    # print("sample :", len(sample), sample)

    # shuffle pos neg
    random.shuffle(sample)

    # return sample in tensor
    sample_tensor = torch.tensor(sample, dtype=torch.float32)
    return sample_tensor

In [13]:
data = rand_sample_generation(1000, 1000)
mis_data = rand_sample_generation_mis50(1000, 1000)

In [14]:
# compare with mislabeled sample

fin_weight, err_rate = pocket_alg(data, max_iter=200)
print("fin_weight:", fin_weight, ", err_rate:", err_rate)
print("pocket accuracy:", 1-err_rate)

fin_weight_mis, err_rate_mis = pocket_alg(mis_data, max_iter=200)
print("fin_weight_mis:", fin_weight_mis, ", err_rate_mis:", err_rate_mis)
print("pocket accuracy (mis) :", 1-err_rate_mis)

fin_weight: tensor([ 658.8668, -943.4258,  289.5936]) , err_rate: 0.0
pocket accuracy: 1.0
fin_weight_mis: tensor([ 71.0908, -29.7757,  13.7411]) , err_rate_mis: 0.1525
pocket accuracy (mis) : 0.8475
